In [1]:
#!pip install soundfile

In [26]:
"""
Adjusted by: Emile Peeters
Author: Hannes Rosseel

This demo file shows how to generate an Exponential Sine Sweep (ESS) and its
inverse filter according to Farina [1] and how to calculate a Room Impulse
Response (RIR) from a measured sweep.

Do not disseminate this code without permission.
"""

###############################################################################
# Import required libraries
###############################################################################
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import os
import test_lib as lib
from scipy.io import wavfile
#from lib import ess_gen_farina, ess_parse_farina

###############################################################################
# Create Exponential Sine Sweep from parameters
###############################################################################
fs = 48000  # Sampling Frequency [Hz]

f1 = 20  # Initial frequency [Hz]
f2 = fs // 2  # Final frequency [Hz]

T_sweep = 10  # Sweep duration [sec.]
T_idle = 3  # Idle duration [sec.]

fade_in = int(0.100 * fs)  # fade-in window of 100 ms [samples]

t_sweep = np.arange(0, (T_sweep + T_idle) * fs) / fs  # Time vector [sec.]

###############################################################################
# Generate exponential sine sweep and its inverse filter
###############################################################################
sweep, inverse = lib.ess_gen_farina(f1, f2, T_sweep, T_idle, fs,
                                fade_in=fade_in,
                                cut_zerocross=True)

###############################################################################
# Define measurement ranges
measurement_ranges = [(4, 15, 'ada18'), (16, 27, 'ada8')]

for measurement_range in measurement_ranges:
    start_measurement, end_measurement, suffix = measurement_range

    rirs = []  # List to store RIRs for each measurement

    for measurement_number in range(start_measurement, end_measurement + 1):
        measured_sweep = []

        for channel in ['A', 'B', 'C', 'D', 'E', 'F']:
            file_path = f'Measurements_Team4/Recordings SPAI/mic{channel}_{str(measurement_number).zfill(3)}.wav'

            if os.path.exists(file_path):
                data, _ = sf.read(file_path)

                # Call ess_parse_farina for each channel
                h_channel = lib.ess_parse_farina(data, inverse, T_sweep, T_idle, fs, causality=True)
                measured_sweep.append(h_channel)

        print(measured_sweep)
        measured_sweep = np.array(measured_sweep).T
        print(measured_sweep.shape)
        print(measured_sweep)
        # Save the combined RIRs as a WAV file using soundfile.write
        output_file = f"/python_exported_rirs/rir{start_measurement}_{suffix}.wav"
        print(f"Saving to: {output_file}")
        try:
            wavfile.write(output_file, fs, measured_sweep)
        except Exception as e:
            print(f"Error writing WAV file: {e}")

    #     # Combine the RIRs along the second axis
    #     rir_measurement = np.vstack(measured_sweep)

    #     # Append the RIRs for the current measurement to the list
    #     rirs.append(rir_measurement)

    # # Combine the RIRs for all measurements along the second axis
    # combined_rirs = np.vstack(rirs)

    # print(combined_rirs.shape)
    # print(combined_rirs)

    

[array([0.00088356, 0.01030538, 0.00382279, ..., 0.00571943, 0.02611363,
       0.0017481 ]), array([ 0.00888019, -0.00101458,  0.00914904, ...,  0.0065709 ,
        0.02705184,  0.00907902]), array([0.00305525, 0.00756598, 0.00823631, ..., 0.00935973, 0.02152146,
       0.00433985]), array([-0.00075678,  0.00934261,  0.0015578 , ...,  0.00212052,
        0.0200549 ,  0.00604383]), array([ 0.00617715,  0.00441845,  0.00517575, ...,  0.00067871,
        0.00456221, -0.00085056]), array([ 4.19277120e-03, -2.62848946e-03,  4.85067223e-03, ...,
        3.65244808e-04,  1.29786988e-03, -7.46844662e-05])]
(144000, 6)
[[ 8.83557381e-04  8.88018875e-03  3.05524699e-03 -7.56783825e-04
   6.17715336e-03  4.19277120e-03]
 [ 1.03053849e-02 -1.01457762e-03  7.56598080e-03  9.34261411e-03
   4.41844925e-03 -2.62848946e-03]
 [ 3.82279148e-03  9.14903507e-03  8.23631124e-03  1.55780061e-03
   5.17575001e-03  4.85067223e-03]
 ...
 [ 5.71942708e-03  6.57090190e-03  9.35973434e-03  2.12051904e-03
   6.78